In [2]:
!nvidia-smi

Sun Mar 19 18:01:25 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 470.74       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    Off  | 00000000:43:00.0 Off |                  Off |
| 30%   36C    P8    31W / 300W |  48111MiB / 48685MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import faiss
assert faiss.get_num_gpus() > 0

In [2]:
import pyterrier as pt
pt.init()

PyTerrier 0.8.0 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)



# Indexing for the Target Corpus

In [ ]:
from pyterrier_colbert.indexing import ColBERTIndexer
index_root="/nfs/indices/BEIR/nfcorpus"
index_name="nfcorpus_colbertIndex"
dataset = pt.get_dataset("irds:beir/nfcorpus")

# default 150, stride 75
checkpoint="http://www.dcs.gla.ac.uk/~craigm/ecir2021-tutorial/colbert_model_checkpoint.zip"

indexer =  pt.text.sliding(text_attr="text", prepend_attr='title') >> ColBERTIndexer(checkpoint,index_root, index_name, chunksize=20)
indexer.index(dataset.get_corpus_iter())

# Indexing for the External Corpus

In [ ]:
from pyterrier_colbert.indexing import ColBERTIndexer
index_root="/nfs/indices/colbert_passage""
index_name="external_colbertIndex"
dataset = pt.get_dataset("trec-deep-learning-passages")
checkpoint="http://www.dcs.gla.ac.uk/~craigm/ecir2021-tutorial/colbert_model_checkpoint.zip"


indexer = ColBERTIndexer(checkpoint,index_root, index_name, chunksize=20)
indexer.index(dataset.get_corpus_iter())

# Loading query set
- Choose the type of query set you would like to load, this demo shows the example code for loading the NFCorpus  query set:
 

In [47]:
topics_nfcorpus=pt.get_dataset("irds:beir/nfcorpus/test").get_topics()
qrels_nfcorpus=pt.get_dataset("irds:beir/nfcorpus/test").get_qrels()
len(topics_nfcorpus)

323

# ColBERT-PRF on Target Corpus

In [44]:
import pyterrier_colbert.ranking
pytcolbertTgt = pyterrier_colbert.ranking.ColBERTFactory("http://www.dcs.gla.ac.uk/~craigm/colbert.dnn.zip", \
                                                      "/nfs/indices/BEIR/nfcorpus", \
                                                      "nfcorpus_colbertIndex",faiss_partitions=100,memtype='mem')
pytcolbertTgt.faiss_index_on_gpu = False
dense_e2eTgt = pytcolbertTgt.set_retrieve() >> pytcolbertTgt.index_scorer(query_encoded=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing ColBERT: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing ColBERT from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing ColBERT from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ColBERT were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['linear.weight']
You should probably TRAI

[May 16, 13:24:23] #> Loading model checkpoint.
[May 16, 13:24:23] #> Loading checkpoint /nfs/xiao/GOOD_MODELS/colbert.dnn
[May 16, 13:24:24] #> checkpoint['epoch'] = 0
[May 16, 13:24:24] #> checkpoint['batch'] = 44500
[May 16, 13:24:25] #> Loading the FAISS index from /nfs/indices/BEIR/nfcorpus/nfcorpus_colbertIndex/ivfpq.100.faiss ..
[May 16, 13:24:25] #> Building the emb2pid mapping..
[May 16, 13:24:25] len(self.emb2pid) = 1064705


Loading index shards to memory:   0%|          | 0/1 [00:00<?, ?shard/s]

Loading reranking index, memtype=mem


Loading index shards to memory: 100%|██████████| 1/1 [00:00<00:00,  1.84shard/s]


In [69]:
from pyterrier_colbert.ranking import ColbertPRF
dense_e2eTgt = pytcolbertTgt.set_retrieve() >> pytcolbertTgt.index_scorer(query_encoded=True)
PRFTgt = dense_e2eTgt %10 \
>> ColbertPRF(pytcolbertTgt, k=24, fb_docs=3, fb_embs=10, beta=1)\

100%|██████████| 30522/30522 [00:00<00:00, 259821.66it/s]


In [70]:
print(len(topics_nfcorpus))
res = PRFTgt(topics_nfcorpus)
import pandas as pd
pd.to_pickle(res,"/path_to_save/tgt/Tgt_BEIR.nfcorpus.embs.pkl")

323


#  ColBERT-PRF on External Corpus

In [17]:
from pyterrier_colbert.ranking import ColBERTFactory

import pyterrier_colbert.ranking
pytcolbert = pyterrier_colbert.ranking.ColBERTFactory("http://www.dcs.gla.ac.uk/~craigm/colbert.dnn.zip", \
                                                      "/nfs/indices/colbert_passage", \
                                                      "external_colbertIndex",faiss_partitions=100,memtype='mem')
pytcolbert.faiss_index_on_gpu = False
dense_e2e = pytcolbert.set_retrieve() >> pytcolbert.index_scorer()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing ColBERT: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing ColBERT from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing ColBERT from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ColBERT were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['linear.weight']
You should probably TRAI

[Apr 24, 13:21:35] #> Loading model checkpoint.
[Apr 24, 13:21:35] #> Loading checkpoint /nfs/xiao/GOOD_MODELS/colbert.dnn
[Apr 24, 13:21:36] #> checkpoint['epoch'] = 0
[Apr 24, 13:21:36] #> checkpoint['batch'] = 44500
[Apr 24, 13:21:37] #> Loading the FAISS index from /nfs/craigm/indices/colbert_passage/index_name3/ivfpq.100.faiss ..
[Apr 24, 13:22:03] #> Building the emb2pid mapping..
[Apr 24, 13:22:42] len(self.emb2pid) = 687989391


Loading index shards to memory:   0%|          | 0/24 [00:00<?, ?shard/s]

Loading reranking index, memtype=mem


Loading index shards to memory: 100%|██████████| 24/24 [02:53<00:00,  7.21s/shard]


In [19]:

PRF_ext = dense_e2e %10 >> ColbertPRF(pytcolbert, k=24, fb_docs=3, fb_embs=10, beta=1)

100%|██████████| 30522/30522 [00:00<00:00, 216550.06it/s]


In [20]:
PRF_ext(topics2019.head(1))

/opt/conda/lib/python3.7/site-packages/pyterrier_colbert/ranking.py:697: UserWarning: index_scorer() used with query_encoded=False, but query_embs column present in input. Should you use query_encoded=True?
  warn("index_scorer() used with query_encoded=False, but query_embs column present in input. Should you use query_encoded=True?")


,qid,docno,query,query_embs,query_toks,query_weights
0,156493,3288601,do goldfish grow,"[[tensor(0.0995), tensor(0.1051), tensor(-0.05...","[bowls, ##´, ##fish, kept, gold, fish, grow, f...","[tensor(1.), tensor(1.), tensor(1.), tensor(1...."


In [ ]:
print(len(topics_nfscorpus))
res = PRF_ext(topics_nfscorpus)
import pandas as pd
pd.to_pickle(res,"/path_to_save/ext/External_BEIR.nfscorpus.embs.pkl")

# External Dense (ColBERTT-PRF) Expansion

In [75]:
import pandas as pd
res_tgt = pd.read_pickle("/path_to_save/tgt/Tgt_BEIR.nfcorpus.embs.pkl")
res_tgtTrans = pt.transformer.SourceTransformer(res_tgt)
res_ext = pd.read_pickle("/path_to_save/ext/External_BEIR.nfscorpus.embs.pkl")
res_extTrans = pt.transformer.SourceTransformer(res_ext)


In [76]:
def transform(res_tgt, res_ext):
    import torch
    first_row = res_ext.iloc[0]
    
    # concatenate the external embeddings to the target query embeddings 
    newemb = torch.cat((res_tgt.iloc[0].query_embs,res_ext.iloc[0].query_embs[32:]),0)

    # the weights column defines important of each query embedding
    newweights = torch.cat((res_tgt.iloc[0].query_weights, res_ext.iloc[0].query_weights[32:]),0)
    newtoks = res_tgt.iloc[0].query_toks+ (res_ext.iloc[0].query_toks)

    # generate the revised query dataframe row
    rtr = pd.DataFrame([
        [first_row.qid, 
         first_row.docno,
         first_row.query, 
         newemb, 
         newtoks, 
         newweights]
        ],
        columns=["qid","docno", "query", "query_embs", "query_toks", "query_weights"])
    return rtr
def mergePRF(res_tgt_input, res_ext_input):
    rtrMerge = pd.DataFrame(columns = ["qid","docno", "query", "query_embs", "query_toks", "query_weights"])
    qidlist = res_ext_input.qid.unique()
    for qid in qidlist:
        res_tgt = res_tgt_input[res_tgt_input["qid"] == qid]
        res_ext = res_ext_input[res_ext_input["qid"] == qid]
        new_query_df = transform(res_tgt, res_ext)
        rtrMerge = rtrMerge.append(new_query_df)
    return rtrMerge

In [77]:

resMerge = mergePRF(res_tgt, res_ext) 
resMerge.head()

,qid,docno,query,query_embs,query_toks,query_weights
0,PLAIN-1008,4557512,deafness,"[[tensor(-0.0544), tensor(0.0031), tensor(0.01...","[##ancy, disability, prevented, ##eti, ##dine,...","[tensor(1.), tensor(1.), tensor(1.), tensor(1...."
0,PLAIN-1018,8568028,dha,"[[tensor(-0.0804), tensor(0.1422), tensor(0.09...","[##gal, dh, ##xa, ##he, al, development, condu...","[tensor(1.), tensor(1.), tensor(1.), tensor(1...."
0,PLAIN-102,1331575,stopping heart disease in childhood,"[[tensor(0.0095), tensor(0.1305), tensor(0.048...","[##care, stat, childhood, eating, ##8, glucose...","[tensor(1.), tensor(1.), tensor(1.), tensor(1...."
0,PLAIN-1028,755539,dietary scoring,"[[tensor(-0.0138), tensor(0.0760), tensor(0.06...","[residues, ##static, pest, sd, score, ##i, 40,...","[tensor(1.), tensor(1.), tensor(1.), tensor(1...."
0,PLAIN-1039,2593757,domoic acid,"[[tensor(0.1494), tensor(0.0146), tensor(0.119...","[dom, wildlife, coma, ##oic, shell, poisoning,...","[tensor(1.), tensor(1.), tensor(1.), tensor(1...."


In [79]:
#resMerge = pd.read_csv(path_resMErge)
MergeTrans = pt.transformer.SourceTransformer(resMerge)
pipeCE2_ranker = (MergeTrans 
                  >> pytcolbertTgt.set_retrieve(query_encoded=True) 
                  >> (pytcolbertTgt.index_scorer(query_encoded=True, add_ranks=True, batch_size=5000)
                  >> pt.text.max_passage() % 1000)

In [80]:

print(len(topics_nfcorpus))
res = pd.concat([resPart for resPart in pipeCE2_ranker.transform_gen(topics_nfcorpus, batch_size=100)])
pt.io.write_results(res, "/path_to_save/CE.nfcorpus.res.gz")
evalMeasuresDict = pt.Utils.evaluate(res,qrels_nfcorpus,metrics=["ndcg_cut_10","ndcg_cut_20","ndcg_cut_1000"])
evalMeasuresDict

323


{'ndcg_cut_10': 0.33206571694278414,
 'ndcg_cut_20': 0.30744815120916363,
 'ndcg_cut_1000': 0.3834315067045998}